<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/BERT_Embedding_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install transformers

In [14]:
!pip install stop-words

In [15]:
# using bert-model to create static the word-embedding
# https://datascience.stackexchange.com/questions/85566/how-pre-trained-bert-model-generates-word-embeddings-for-out-of-vocabulary-words

from distutils.util import split_quoted
import torch
from transformers import BertTokenizer, BertModel
from sklearn.datasets import fetch_20newsgroups
import re
from nltk.tokenize import word_tokenize
import nltk
import string
nltk.download('punkt')

#with open('src/stops.txt', 'r') as f:
#    stops = f.read().split('\n')

from stop_words import get_stop_words
stop_words = get_stop_words('en')
    

newsgroups_train = fetch_20newsgroups(subset='train')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [32]:
# read train_data from 20newsgroups
def read_raw_documents():
    raw_documents = []
    raw_labels = []
    for i in range(0,len(newsgroups_train.data)):
        raw_documents.append(newsgroups_train.data[i])
        raw_labels.append(newsgroups_train.target[i])
    return raw_documents, raw_labels

def simple_preprocess(raw_documents):
    def only_letters(tested_string):
        for letter in tested_string:
            if letter not in "abcdefghijklmnopqrstuvwxyz":
                return False
        return True
    def clean_doc_for_bert(doc): 
        doc = doc.replace(">","").lower()
        word_list = word_tokenize(doc) #only using empty space and punctation for tokenization
        cleaned = []
        for w in word_list:
            if w not in stop_words:
                if w in string.punctuation or only_letters(w): #using only character from punctation and alpha characters
                    if w in string.punctuation or len( set(w) ) > 1: #punctation with len 1 allowed but alpha word must be longer then 1
                        cleaned.append( w)
        return " ".join(cleaned), cleaned  #save doc in string and in token-list         
       
    cleaned_documents = []
    for doc in raw_documents:
        doc_in_string, doc_in_token_list = clean_doc_for_bert(doc)
        cleaned_documents.append(doc_in_string)
    return cleaned_documents

def transform_to_sentences_with_labels():
    # we will not use labels
    sentences_with_labels = []
    return sentences_with_labels

def fine_tune_bert():
    # should to be trained?
    # no, because for topic modelling, that is usupervised problem. We just find topics for the documents
    # topic modelling no targets
    return True

def transform_to_sentences(docs): #no labels
    data_as_sentences = []
    for doc in docs:
      for sent in doc.split(". "): #make sentences
        updated_sent = " ".join([t for t in sent.split(" ") if len(t) > 1])
        if len(updated_sent.split(" ")) > 1:
            data_as_sentences.append(updated_sent)
        else:
            if updated_sent not in data_as_sentences:
                data_as_sentences.append(updated_sent)
    return data_as_sentences

def split_long_sentence(splitted_sent, given_len):
    subsents = []
   #for i in range(0,len(splitted_sent), given_len):
    i=0
    while i < len(splitted_sent): 
        if i == 0:
            sub = " ".join(splitted_sent[i:i+given_len])
            subsents.append(sub)
            i = i + given_len
        if i!=0:
            j = i + given_len - 5 #windown 5
            if j + given_len <= len(splitted_sent):
                sub = " ".join(splitted_sent[j:j + given_len])
                subsents.append(sub)
            else:
                sub = " ".join(splitted_sent[j:])
                if len(sub)>1:
                    subsents.append(sub)
            i = j + given_len
    return subsents

def handle_long_sentences(sentences, given_len):
    # overlapped splitting sentence windown 5
    subsents = []
    deleted_long_sents = []
    for sent in sentences:
        splitted_sent = sent.split(" ")
        if len(splitted_sent) > given_len:
          long_sent_subsents = split_long_sentence(splitted_sent, given_len)
          subsents.extend(long_sent_subsents)
          deleted_long_sents.append(sent)
    # update sentences: remove and add subsents
    for del_sent in deleted_long_sents:
        sentences.remove(del_sent)
    for add_sent in subsents:
        sentences.append(add_sent)
    return sentences

def create_marked_senteces(sentences):
    return ['[CLS] ' + sent.strip() + ' [SEP]' for sent in sentences]
def save_sents_to_txt(shorted_sentences):
    with open(r'./bert_sentences.txt', 'w') as fp:
      for sent in shorted_sentences:
          # write each item on a new line
          fp.write(f'{sent} \n')
      print('saving sentences from bert-processing')
    return True

# **Datenvorbereitung**

In [33]:
print("reading data:...")
raw_documents, _ = read_raw_documents()
print(len(raw_documents))
print("preprocess data:...")
preprocessed_docs = simple_preprocess(raw_documents)
print(len(preprocessed_docs))
print("transform to sentences:...")
sentences = transform_to_sentences(preprocessed_docs)
print("split sentences to 128 tokens:...")
shorted_sentences =  handle_long_sentences(sentences, 128)
shorted_sentences = create_marked_senteces(shorted_sentences)
# write sentences to txt files
with open(r'bert_sentences.txt', 'w') as fp:
    for sent in shorted_sentences:
        # write each item on a new line
        fp.write(f'{sent} \n')
    print('saving sentences from bert-processing')
print("finished: ...")



reading data:...
11314
preprocess data:...
11314
transform to sentences:...
split sentences to 128 tokens:...
saving sentences from bert-processing
finished: ...


# **Embeddings**

In [47]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [66]:
def tokenizer_for_a_sent(sent):
  this_sent_tokenizer = tokenizer(sent)
  # index of token in the vocabulary
  indexed_tokens = this_sent_tokenizer.input_ids
  segments_ids = [1] * len(indexed_tokens)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  return tokens_tensor, segments_tensors

In [69]:
for i in range(0,len(sentences)):
  sent = shorted_sentences[i]
  tokens_tensor, segments_tensors = tokenizer_for_a_sent(sent)
  """
  tk = tokenizer(sent)
  print(tk.input_ids)
  print(tk.token_type_ids)
  print(tk.word_ids())
  print(len(tk.word_ids()))
  print(tokenizer.tokenize(sent))
  print(len(tokenizer.tokenize(sent)))
  print("---------------------------------------")
  """
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
    #print(hidden_states)
    print(len(hidden_states))
    print(hidden_states[0].shape)
  break

13
torch.Size([1, 25, 768])


In [ ]:
def get_token_embedding_from_sent(belonged_embeddings):
  # average of belongend-embeddings
  return torch.mean(belonged_embeddings, dim = 0)
def find_belonged_embeddings_of_token(token, tokenized_text, embeddings_outputs):
  belonged_embeddings = []
  belonged_indices = []
  for idx, t in enumerate(tokenized_text):
    if t.strip("#") in token:
      belonged_indices.append(i)
  for idx in belonged_indices:
    belonged_embeddings.append(embeddings_outputs[i])
  return belonged_embeddings
def get_embeddings_in_sent(sent_in_token_list, embeddings_outputs):
  return